# Einführung in Neuronale Netze

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3d/Neural_network.svg" alt="Ein Neuronales Netz (vereinfacht)" width="400px" />

## EINLEITUNGSTEXT
...

## Das einzelne Neuron
<img src="https://miro.medium.com/max/856/1*O7YSSqlOdQuNgMrH7J_3dg.png" alt="Das einzelne Neuron" />

Ein Neuronales Netz besteht aus vielen Neuronen, die miteinander verbunden sind. Dabei sind die Neuronen denen im Gehirn nachempfunden, welche untereinander verbunden sind und die aufeinander reagieren. Bei künstlichen Neuronalen Netzen hat jedes Neuron eine oder mehrere Eingaben $x_1...x_m$ und eine Ausgabe $\^{o_j}$. Die Eingaben werden mit Gewichten $w_1...w_m$ multipliziert und anschließend addiert ($ net = \displaystyle\sum_{i=1}^{m} (x_i \cdot w_i) $). Das Ergebnis wird dann in eine Aktivierungsfunktion $\varphi_j(x)$ ($j$ steht für ein bestimmtes Neuron. Jedes Neuron kann eine eigene Aktivierungsfunktion haben) gegeben, welche das Ergebnis in die Ausgabe des Neurons umwandelt $o_j = \varphi_j(net)$ und die dann ein Eingabewert (ein anderes $x$) für die nächsten Neuronen ist (oder die Ausgabe des Neuronalen Netzes darstellt, wenn das Neuron an der hintersten Ebene ist).